In [ ]:
#| default_exp utils.asian_1x2_pnl

In [ ]:
#| hide

from IPython.core.debugger import set_trace

%load_ext autoreload
%autoreload 2

# 1X2 and Asian Handicap Profit
> Profit for 1X2 and Asian handicap betting

In [ ]:
#| export

import mongoengine
import numexpr
import numpy as np
import pandas as pd

Using the `config.toml` credentials included in the main repository, we must first load games data stored in our MongoDb Cluster before testing our betting environment.

In [ ]:
from betting_env.config.mongo import mongo_init
from betting_env.datastructure.fixtures import Fixture

In [ ]:
# Initialise connections.
mongo_init("prod_atlas")

# Get only one fixture.
fixture = Fixture.get_all_fixtures(5)

# Fixture.
fixture = pd.DataFrame(fixture.as_pymongo())
fixture.head()

,_id,gameId,optaGameId,gameDate,homeTeamId,homeTeamOptaId,homeTeamName,awayTeamId,awayTeamOptaId,awayTeamName,...,awayTeamLineupSlots,awayTeamFormation,preGameOdds1,preGameOddsX,preGameOdds2,preGameAhHome,preGameAhAway,lineId,result,postGameGd
0,63fbf2a515a225c64d74249d,219ef70c0e8a803ec1efdb793443edfaa32398690c7829...,991003,2018-08-22T18:45:00.000Z,aeb2f56fcedbcf4cd5c780179766996c7bf0b308064541...,5,Blackburn Rovers,f8daf96ad35eebf1c0a5886c72734ba7dec366d6637052...,108,Reading,...,"[4, 2, 8, 6, 5, 3, 1, 7, 10, 11, 9]",4-4-2,1.980,3.36,4.51,1.90,2.02,-0.5,1.0,0
1,63fbf2a515a225c64d74249c,174dba7291174b4dbbfa9ea12dd944bb45bdd8ed905524...,990997,2018-08-22T18:45:00.000Z,126905d14981e6b97912ad4fec354035ccef26cb8ec4e1...,7,Aston Villa,419088133137a53bfdb1b7e2e682d223d33a6fa075bbfe...,94,Brentford,...,"[1, 5, 9, 8, 3, 6, 7, 10, 4, 11, 2]",4-2-3-1,2.620,3.53,2.77,1.86,2.06,0.0,1.0,0
2,63fbf2a515a225c64d74249f,0655e244d8d596b5572e86426e2a7ca6178044efa59437...,991013,2018-08-25T14:00:00.000Z,9ee012a80cade2df55b71580bf5e238bcd6be6f696fdc1...,45,Norwich City,38ca605bcd29a5a37697ca66e533ae817ced71b6bf275c...,2,Leeds United,...,"[8, 2, 9, 4, 1, 5, 10, 7, 3, 11, 6]",4-1-4-1,2.620,3.60,2.77,1.92,2.02,0.0,2.0,-3
3,63fbf2a515a225c64d74249e,019c223b4a03917c2f1685beab4d5d278f7bff3913f239...,991018,2018-08-25T14:00:00.000Z,eb89c068ca204a72408360450847a990c97c5b5ff0ec9f...,110,Stoke City,bbb63e4ea54b0d60b48a1f8440254d7e656dfbfcbef825...,88,Hull City,...,"[2, 6, 1, 4, 8, 9, 10, 3, 7, 11, 5]",4-4-1-1,1.917,3.48,4.62,1.88,2.04,-0.5,0.0,2
4,63fbf2a515a225c64d7424a0,0f9ad12eec9f24277ab491f5f26f610eaa918903a34147...,991014,2018-08-25T16:30:00.000Z,04c71986b6503ba5b09a7098ceb79954d20049f21ba45b...,17,Nottingham Forest,95d3bddc19a15d34a7876dcffc1a3e9bc63d809b69308a...,41,Birmingham City,...,"[3, 2, 4, 9, 11, 8, 7, 1, 5, 6, 10]",4-4-2,2.040,3.41,4.12,2.02,1.90,-0.5,1.0,0


## Popular betting markets

There are many types of bets available in football today but betting on the match outcome, known as the `1x2` market, is the most popular one by far. The punter can bet the 3 possible match outcomes: *Home-win*, *draw* or *away-win*. 

A second popular market, in particular in Asia, is known as Asian handicap and is directly linked to the game goal difference. It offers only 2 betting options known as *home* (1) or *away* (2) and assumes that one of the team will start with a handicap (known as the line) and the other with an advantage. There are 3 types of Asian handicap markets:

  - *Integer Lines*: if the handicap chosen is equal to the score difference, it can result in a bet being returned.
  - *Half Lines*: the outcome of the wager is either a win or a loss.
  - *Quarter Lines*: Divide your wager equally between the two AH lines above and beyond. For instance, a handicap of -0.75 is really two handicaps for the price of one, with half of your bet being made at -0.5 and the other half at -1, both of which are given the same odds. In this case, we have a win, half-win, half-loss, or a loss.

## PnL computation

### 1X2 market

The PnL of the 1x2 market depends on the match outcome (home-win/draw/away-win) and is *binary*: the punter can either make a profit and get his stake returned or loose his stake.

$$ profit_{1X2} =  Stake *(Odds \times -1)$$
With:

+ Stake : Invested money
+ Odds : 1X2 European/decimal betting odds (> 1)

In [ ]:
#| export


def pnl_1X2(
    selection: np.ndarray,  # The amount invested on each selection.
    outcome: np.ndarray,  # Game result (Binary side outcome), shape=(1,5).
    odds: np.ndarray,  # odds for the current game, shape=(1,5)
) -> np.ndarray:  # 1X2 PnL
    "Returns the 1X2 PnL."
    assert selection.shape == odds.shape, "odds and selection should be same shape!"
    assert outcome.shape == odds.shape, "odds and outcome should be same shape!"
    n = selection.shape[0]

    pnl = selection[:, :3] * outcome[:, :3] * odds[:, :3] - selection[:, :3]
    return pnl.sum(axis=1).reshape((n, -1))

In [ ]:
# Bet action (here, our action is betting 20% of our balance on away team).
selection = (
    np.array([[0, 0, 0.2, 0, 0], [0, 0.7, 0, 0, 0], [0.8, 0, 0.0, 0, 0]]) * 100.0
)

info = fixture.head(3)

# Game odds.
odds = info[
    [
        "preGameOdds1",
        "preGameOddsX",
        "preGameOdds2",
        "preGameAhHome",
        "preGameAhAway",
    ]
].values

# Game result index(homewin -> 0 , draw -> 1, awaywin -> 2).
game_result = info["result"].values.astype(int)
binary_result = np.zeros_like(selection)
np.put_along_axis(binary_result, np.expand_dims(game_result, axis=1), 1, axis=1)

pnl_1X2(selection, binary_result, odds)

array([[-20. ],
       [177.1],
       [-80. ]])

### Asian handicap market

The PnL for the Asian handicap line is more complicated. It is related to the game goal-difference and the actual handicap line. It can result, as we described above, in 5 outcomes: win/half-win/fold/half-loss/loss.

$$ profit_{AH} = \begin{cases} 
     0.0 & \text{If Gd + line = 0} \\\\ 
     \frac{odds -1}{2}& \text{If Gd + line = 0.25}\\\\
     odds -1 & \text{If Gd + line >= 0.5} \\\\
     -0.5 & \text{If Gd + line = -0.25} \\\\
     -1.0 & \text{If Gd + line <= -0.5} \end{cases} $$
 
 With:

+ Gd : game goal difference
+ line: Asian handicap line
+ Odds : Asian handicap odds

In [ ]:
#| export


def pnl_ah(
    selection: np.ndarray,  # The amount invested on each selection; shape n x 5; last 2 are for home/away asian handicap
    odds: np.ndarray,  # market odds in 1|X|2|A1|A2 order; shape n x 5
    obs_gd: np.ndarray,  # Game goal-difference; shape (n,)
    ah_line: np.ndarray,  # Asian line could be integer, half or quarter line; shape (n,)
) -> np.ndarray:  # Asian Handicap PnL.
    "Returns the Asian Handicap PnL"

    # check dimension
    n = selection.shape[0]
    obs_gd = obs_gd.reshape((n, -1))
    ah_line = ah_line.reshape((n, -1))
    assert selection.shape == odds.shape, "odds and selection should be same shape!"

    def _pnl_ah(obs_gd, ah_line, ah_odds):
        "provides the asian outcome given for a unit bet."
        # Team advantage.
        gd_advantage = obs_gd + ah_line

        if gd_advantage == 0:
            return 0.0
        elif gd_advantage == 0.25:
            return (ah_odds - 1) * 0.5
        elif gd_advantage == -0.25:
            return -0.5
        elif gd_advantage >= 0.5:
            return ah_odds - 1
        elif gd_advantage <= -0.5:
            return -1.0

    ah_selection = selection[:, -2:]
    ah_odds = odds[:, -2:]

    # change line sign if betting on away
    ah_idx = np.where(ah_selection > 0)
    flip_sign = np.zeros_like(ah_line)
    flip_sign[ah_idx[0], 0] = np.where(ah_idx[1] == 0, 1, -1)

    # reshape odds
    _ah_odds = np.zeros_like(ah_line)
    _ah_odds[ah_idx[0], 0] = ah_odds[ah_idx]

    # reshape selection
    _ah_sel = np.zeros_like(ah_line)
    _ah_sel[ah_idx[0], 0] = ah_selection[ah_idx]

    _pnl_ah_v = np.vectorize(_pnl_ah)
    _unit_pnl = _pnl_ah_v(
        obs_gd * flip_sign, ah_line * flip_sign, _ah_odds.reshape((n, -1))
    )
    
    return _unit_pnl * _ah_sel.reshape((n, -1))

In [ ]:
selection = (
    np.array([[0, 0, 0.0, 0.0, 0.5], [0, 0, 0, 0, 0], [0, 0, 0.0, 0.2, 0]]) * 100.0
)

info = fixture.head(3)

# Game odds.
odds = info[
    [
        "preGameOdds1",
        "preGameOddsX",
        "preGameOdds2",
        "preGameAhHome",
        "preGameAhAway",
    ]
].values

obs_gd = info.postGameGd.values
ah_line = info.lineId.values


pnl_ah(selection, odds, obs_gd, ah_line)

array([[ 51.],
       [  0.],
       [-20.]])

## Pnl wrapper

This function computes the total PnL:

In [ ]:
#| export

def pnl(
    selection: np.ndarray,  # The amount invested on each selection; shape n x 5; last 2 are for home/away asian handicap
    odds: np.ndarray,  # market odds in 1|X|2|A1|A2 order; shape n x 5
    obs_gd: np.ndarray,  # Game goal-difference; shape (n,)
    ah_line: np.ndarray,  # Asian line could be integer, half or quarter line; shape (n,)
) -> np.ndarray:  # Asian Handicap PnL.
    "Returns the total PnL"
    # check dimesnion
    n = selection.shape[0]
    assert selection.shape == odds.shape, "odds and selection should be same shape!"

    if len(obs_gd.shape) > 1 and obs_gd.shape[1] == 1:
        obs_gd = obs_gd.squeeze(1)
    binary_result = np.zeros_like(selection)
    binary_result[obs_gd > 0, 0] = 1
    binary_result[obs_gd == 0, 1] = 1
    binary_result[obs_gd < 0, 2] = 1

    # 1x2 pnl
    _pnl_1x2 = pnl_1X2(selection, binary_result, odds)

    # ah-line
    _pnl_ah = pnl_ah(selection, odds, obs_gd, ah_line)
    
    _total_pnl = _pnl_1x2 + _pnl_ah

    return _total_pnl

In [ ]:
selection = (
    np.array(
        [
            [0, 0.0, 0, 0.0, 0.3],  # bet on asian 2
            #[0.3, 0, 0, 0, 0],  # bet on home
            #[0, 0, 0.6, 0.0, 0],  # bet on away
            #[0, 0, 0.0, 0.6, 0],  # bet on asian 1
            #[0, 0.5, 0.0, 0.0, 0],  # bet on X
        ]
    )
    * 100.0
)

info = fixture.head(selection.shape[0])

# Game odds.
odds = info[
    [
        "preGameOdds1",
        "preGameOddsX",
        "preGameOdds2",
        "preGameAhHome",
        "preGameAhAway",
    ]
].values

obs_gd = info.postGameGd.values
ah_line = info.lineId.values

pnl(selection, odds, obs_gd, ah_line)

array([[30.6]])

In [ ]:
#| hide

import nbdev

nbdev.nbdev_export()